In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# dependencies
connect to cpu if want to use behaviour cloning

check learning rate for both PPO and TRPO

check timesteps required and set all cases with the same upper limit

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 27.3 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823

# environment and expert

In [4]:
import gymnasium as gym
import imitation

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
SEED = 42

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def make_env_expert_rollouts(n_trajs):
    # Create the environment, load expert policy, and sample trajectories
    # from the expert policy and environment.
    global SEED
    env = make_vec_env(
    "seals:seals/HalfCheetah-v1", # seals:seals/CartPole-v0
    rng=np.random.default_rng(SEED),
    n_envs=n_trajs,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
    )
    expert = load_policy(
        "ppo-huggingface",
        organization="HumanCompatibleAI",
        env_name="seals/HalfCheetah-v1",
        venv=env,
    )
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=1000, min_episodes = n_trajs),
        rng=np.random.default_rng(SEED), #The random state to use for sampling trajectories
    )

    return env, expert, rollouts


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
import torch
def make_PPO_learner_and_trainer(env, expert, rollouts):
    global SEED

    learner_PPO = PPO(
        env=env,
        policy=MlpPolicy,
        batch_size=64,
        clip_range=0.1,
        ent_coef=3.992371122209408e-6,
        learning_rate=0.00026250519057717037,
        gae_lambda=0.95,
        gamma=0.95,
        n_epochs=5,
        seed=SEED,
        vf_coef=0.11483689492120866,
    )
    load_from_saved_policy = torch.load("BC_MlpPolicy_HalfCheetah_1.pth.tar")
    learner_PPO.policy.load_state_dict(load_from_saved_policy.state_dict(), strict=False)

    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
    )
    gail_trainer = GAIL(
        demonstrations = rollouts, # expert trajectories
        demo_batch_size = 1000,
        gen_replay_buffer_capacity = 512,
        n_disc_updates_per_round = 8, #
        venv = env,
        gen_algo = learner_PPO,
        reward_net=reward_net,
        allow_variable_horizon=True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,
    )
    return learner_PPO, gail_trainer

# GAIL-PPO 1 trajectory - 1 traj BC

In [19]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)

In [36]:
learner_PPO_1, gail_trainer_PPO_1 = make_PPO_learner_and_trainer(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [12]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1675.82481005 +/- 38.96984282937923


In [37]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [23]:
gail_trainer_PPO_1.train(300000)
# An upper bound on the number of transitions to sample from the environment during training.

round:   0%|          | 0/146 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 1.33e+03 |
|    gen/time/fps             | 465      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 4        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.521    |
|    disc/disc_acc_expert             | 0.292    |
|    disc/disc_acc_gen                | 0.749    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.692    |
|    disc/disc_proportion_expert_pred | 0.272    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1e+03    |
|    disc/n_generated                 | 1e+03    |
-

round:   1%|          | 1/146 [00:05<12:36,  5.21s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.36e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 1.29e+03   |
|    gen/time/fps                    | 467        |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 4          |
|    gen/time/total_timesteps        | 4096       |
|    gen/train/approx_kl             | 0.07733267 |
|    gen/train/clip_fraction         | 0.699      |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 4.18       |
|    gen/train/explained_variance    | -0.0699    |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 6.48       |
|    gen/train/n_updates             | 5          |
|    gen/train/policy_gradient_loss  | 0.0191     |
|    gen/train/std                   | 0.13       |
|    gen/tra

round:   1%|▏         | 2/146 [00:10<12:30,  5.21s/it]

...

round:  99%|█████████▊| 144/146 [12:47<00:10,  5.30s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 1.13e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 667         |
|    gen/time/fps                    | 458         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 4           |
|    gen/time/total_timesteps        | 296960      |
|    gen/train/approx_kl             | 0.078434736 |
|    gen/train/clip_fraction         | 0.687       |
|    gen/train/clip_range            | 0.1         |
|    gen/train/entropy_loss          | 3.55        |
|    gen/train/explained_variance    | 0.944       |
|    gen/train/learning_rate         | 0.000263    |
|    gen/train/loss                  | 0.207       |
|    gen/train/n_updates             | 720         |
|    gen/train/policy_gradient_loss  | 0.0209      |
|    gen/train/std                   | 0.142  

round:  99%|█████████▉| 145/146 [12:52<00:05,  5.31s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 1.12e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 664         |
|    gen/time/fps                    | 457         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 4           |
|    gen/time/total_timesteps        | 299008      |
|    gen/train/approx_kl             | 0.053922832 |
|    gen/train/clip_fraction         | 0.65        |
|    gen/train/clip_range            | 0.1         |
|    gen/train/entropy_loss          | 3.56        |
|    gen/train/explained_variance    | 0.726       |
|    gen/train/learning_rate         | 0.000263    |
|    gen/train/loss                  | 0.285       |
|    gen/train/n_updates             | 725         |
|    gen/train/policy_gradient_loss  | 0.0198      |
|    gen/train/std                   | 0.142  

round: 100%|██████████| 146/146 [12:58<00:00,  5.33s/it]


In [24]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [38]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)


Rewards before training: 1154.3682143500002 +/- 439.48203413760854


In [25]:
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards after training: 1273.6559433500001 +/- 344.41345653692446


# GAIL-PPO 4 trajectory - 1 traj BC

In [33]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_PPO_4, gail_trainer_PPO_4 = make_PPO_learner_and_trainer(env_4, expert_4, rollouts_4)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [27]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1665.0737670499998 +/- 36.99955227313197


In [34]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [29]:
gail_trainer_PPO_4.train(300000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/36 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 1.04e+03 |
|    gen/time/fps             | 1573     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 8192     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.505    |
|    disc/disc_acc_expert             | 0.294    |
|    disc/disc_acc_gen                | 0.716    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.694    |
|    disc/disc_proportion_expert_pred | 0.289    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1e+03    |
|    disc/n_generated                 | 1e+03    |
-

round:   3%|▎         | 1/36 [00:08<04:42,  8.07s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.05e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 1.17e+03   |
|    gen/time/fps                    | 1543       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 5          |
|    gen/time/total_timesteps        | 16384      |
|    gen/train/approx_kl             | 0.09128544 |
|    gen/train/clip_fraction         | 0.741      |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 4.15       |
|    gen/train/explained_variance    | -0.0689    |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 1.33       |
|    gen/train/n_updates             | 5          |
|    gen/train/policy_gradient_loss  | 0.0581     |
|    gen/train/std                   | 0.131      |
|    gen/tra

round:   6%|▌         | 2/36 [00:16<04:36,  8.13s/it]

...

round:  94%|█████████▍| 34/36 [04:36<00:16,  8.12s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.15e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 1.06e+03   |
|    gen/time/fps                    | 1595       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 5          |
|    gen/time/total_timesteps        | 286720     |
|    gen/train/approx_kl             | 0.05857006 |
|    gen/train/clip_fraction         | 0.686      |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 3.44       |
|    gen/train/explained_variance    | 0.993      |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 0.173      |
|    gen/train/n_updates             | 170        |
|    gen/train/policy_gradient_loss  | 0.0376     |
|    gen/train/std                   | 0.145      |
|    gen/tra

round:  97%|█████████▋| 35/36 [04:45<00:08,  8.18s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.16e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 1.1e+03    |
|    gen/time/fps                    | 1564       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 5          |
|    gen/time/total_timesteps        | 294912     |
|    gen/train/approx_kl             | 0.06382537 |
|    gen/train/clip_fraction         | 0.695      |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 3.43       |
|    gen/train/explained_variance    | 0.99       |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 0.236      |
|    gen/train/n_updates             | 175        |
|    gen/train/policy_gradient_loss  | 0.04       |
|    gen/train/std                   | 0.145      |
|    gen/tra

round: 100%|██████████| 36/36 [04:53<00:00,  8.15s/it]


30% 1.21e3

In [30]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [35]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: 1382.5016307 +/- 143.16806294215954


In [31]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 1316.9725449500002 +/- 164.748480678671


# GAIL-PPO 10 trajectory - 1 traj BC

In [39]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_PPO_10, gail_trainer_PPO_10 = make_PPO_learner_and_trainer(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1681.11066565 +/- 39.92702650567257


In [40]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [42]:
gail_trainer_PPO_10.train(300000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/14 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 1.21e+03 |
|    gen/time/fps             | 3116     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 6        |
|    gen/time/total_timesteps | 20480    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.567    |
|    disc/disc_acc_expert             | 0.352    |
|    disc/disc_acc_gen                | 0.782    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.688    |
|    disc/disc_proportion_expert_pred | 0.285    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1e+03    |
|    disc/n_generated                 | 1e+03    |
-

round:   7%|▋         | 1/14 [00:13<02:56, 13.57s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.18e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 1.24e+03   |
|    gen/time/fps                    | 3114       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 40960      |
|    gen/train/approx_kl             | 0.10503099 |
|    gen/train/clip_fraction         | 0.743      |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 4.16       |
|    gen/train/explained_variance    | -0.0711    |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 0.404      |
|    gen/train/n_updates             | 5          |
|    gen/train/policy_gradient_loss  | 0.0706     |
|    gen/train/std                   | 0.13       |
|    gen/tra

round:  14%|█▍        | 2/14 [00:27<02:42, 13.55s/it]

...

round:  86%|████████▌ | 12/14 [02:43<00:27, 13.67s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 1.03e+03    |
|    gen/rollout/ep_rew_wrapped_mean | 862         |
|    gen/time/fps                    | 3169        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 6           |
|    gen/time/total_timesteps        | 266240      |
|    gen/train/approx_kl             | 0.067876495 |
|    gen/train/clip_fraction         | 0.716       |
|    gen/train/clip_range            | 0.1         |
|    gen/train/entropy_loss          | 3.75        |
|    gen/train/explained_variance    | 0.917       |
|    gen/train/learning_rate         | 0.000263    |
|    gen/train/loss                  | 0.223       |
|    gen/train/n_updates             | 60          |
|    gen/train/policy_gradient_loss  | 0.0573      |
|    gen/train/std                   | 0.138  

round:  93%|█████████▎| 13/14 [02:57<00:13, 13.63s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 989         |
|    gen/rollout/ep_rew_wrapped_mean | 952         |
|    gen/time/fps                    | 3091        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 6           |
|    gen/time/total_timesteps        | 286720      |
|    gen/train/approx_kl             | 0.070633985 |
|    gen/train/clip_fraction         | 0.703       |
|    gen/train/clip_range            | 0.1         |
|    gen/train/entropy_loss          | 3.72        |
|    gen/train/explained_variance    | 0.958       |
|    gen/train/learning_rate         | 0.000263    |
|    gen/train/loss                  | 0.131       |
|    gen/train/n_updates             | 65          |
|    gen/train/policy_gradient_loss  | 0.0505      |
|    gen/train/std                   | 0.138  

round: 100%|██████████| 14/14 [03:11<00:00, 13.65s/it]


In [43]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [45]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: 1249.4715026000001 +/- 328.3747051152187


In [44]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 999.3148263999999 +/- 445.2257502402302


# give trainer 10 traj BC network

In [46]:
import torch
def make_PPO_learner_and_trainer_10(env, expert, rollouts):
    global SEED

    learner_PPO = PPO(
        env=env,
        policy=MlpPolicy,
        batch_size=64,
        clip_range=0.1,
        ent_coef=3.992371122209408e-6,
        learning_rate=0.00026250519057717037,
        gae_lambda=0.95,
        gamma=0.95,
        n_epochs=5,
        seed=SEED,
        vf_coef=0.11483689492120866,
    )
    load_from_saved_policy = torch.load("BC_MlpPolicy_HalfCheetah_10.pth.tar")
    learner_PPO.policy.load_state_dict(load_from_saved_policy.state_dict(), strict=False)

    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
    )
    gail_trainer = GAIL(
        demonstrations = rollouts, # expert trajectories
        demo_batch_size = 1000,
        gen_replay_buffer_capacity = 512,
        n_disc_updates_per_round = 8, #
        venv = env,
        gen_algo = learner_PPO,
        reward_net=reward_net,
        allow_variable_horizon=True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,
    )
    return learner_PPO, gail_trainer

# GAIL-PPO 1 trajectory - 10 traj BC

In [47]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)

In [48]:
learner_PPO_1, gail_trainer_PPO_1 = make_PPO_learner_and_trainer_10(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [49]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1675.82481005 +/- 38.96984282937923


In [50]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [51]:
gail_trainer_PPO_1.train(300000)
# An upper bound on the number of transitions to sample from the environment during training.

round:   0%|          | 0/146 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 1.5e+03  |
|    gen/time/fps             | 454      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 4        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.499    |
|    disc/disc_acc_expert             | 0.286    |
|    disc/disc_acc_gen                | 0.711    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.695    |
|    disc/disc_proportion_expert_pred | 0.287    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1e+03    |
|    disc/n_generated                 | 1e+03    |
-

round:   1%|          | 1/146 [00:05<12:59,  5.38s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.48e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 1.41e+03   |
|    gen/time/fps                    | 452        |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 4          |
|    gen/time/total_timesteps        | 4096       |
|    gen/train/approx_kl             | 0.26119316 |
|    gen/train/clip_fraction         | 0.76       |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 4.1        |
|    gen/train/explained_variance    | -0.0422    |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 9.38       |
|    gen/train/n_updates             | 5          |
|    gen/train/policy_gradient_loss  | 0.0482     |
|    gen/train/std                   | 0.145      |
|    gen/tra

round:   1%|▏         | 2/146 [00:10<12:56,  5.39s/it]

...

round:  99%|█████████▊| 144/146 [12:55<00:10,  5.46s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.37e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 691        |
|    gen/time/fps                    | 461        |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 4          |
|    gen/time/total_timesteps        | 296960     |
|    gen/train/approx_kl             | 0.08015305 |
|    gen/train/clip_fraction         | 0.68       |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 3.27       |
|    gen/train/explained_variance    | 0.783      |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 0.792      |
|    gen/train/n_updates             | 720        |
|    gen/train/policy_gradient_loss  | 0.00994    |
|    gen/train/std                   | 0.152      |
|    gen/tra

round:  99%|█████████▉| 145/146 [13:01<00:05,  5.42s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.39e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 696        |
|    gen/time/fps                    | 458        |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 4          |
|    gen/time/total_timesteps        | 299008     |
|    gen/train/approx_kl             | 0.08169004 |
|    gen/train/clip_fraction         | 0.698      |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 3.27       |
|    gen/train/explained_variance    | 0.956      |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 0.248      |
|    gen/train/n_updates             | 725        |
|    gen/train/policy_gradient_loss  | 0.0169     |
|    gen/train/std                   | 0.152      |
|    gen/tra

round: 100%|██████████| 146/146 [13:06<00:00,  5.39s/it]


In [52]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [53]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)


Rewards before training: 1573.05025745 +/- 114.37734854220133


In [54]:
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards after training: 1514.27135925 +/- 184.52300006507687


# GAIL-PPO 4 trajectory - 10 traj BC

In [55]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_PPO_4, gail_trainer_PPO_4 = make_PPO_learner_and_trainer_10(env_4, expert_4, rollouts_4)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [56]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1665.0737670499998 +/- 36.99955227313197


In [57]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [58]:
gail_trainer_PPO_4.train(300000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/36 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 1.47e+03 |
|    gen/time/fps             | 1571     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 8192     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.516    |
|    disc/disc_acc_expert             | 0.281    |
|    disc/disc_acc_gen                | 0.752    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.694    |
|    disc/disc_proportion_expert_pred | 0.265    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1e+03    |
|    disc/n_generated                 | 1e+03    |
-

round:   3%|▎         | 1/36 [00:08<04:44,  8.14s/it]

--------------------------------------------------
| raw/                               |           |
|    gen/rollout/ep_len_mean         | 1e+03     |
|    gen/rollout/ep_rew_mean         | 1.49e+03  |
|    gen/rollout/ep_rew_wrapped_mean | 1.4e+03   |
|    gen/time/fps                    | 1588      |
|    gen/time/iterations             | 1         |
|    gen/time/time_elapsed           | 5         |
|    gen/time/total_timesteps        | 16384     |
|    gen/train/approx_kl             | 1.1482937 |
|    gen/train/clip_fraction         | 0.769     |
|    gen/train/clip_range            | 0.1       |
|    gen/train/entropy_loss          | 4.08      |
|    gen/train/explained_variance    | -0.0448   |
|    gen/train/learning_rate         | 0.000263  |
|    gen/train/loss                  | 2.06      |
|    gen/train/n_updates             | 5         |
|    gen/train/policy_gradient_loss  | 0.0719    |
|    gen/train/std                   | 0.145     |
|    gen/train/value_loss      

round:   6%|▌         | 2/36 [00:16<04:34,  8.08s/it]

...

round:  94%|█████████▍| 34/36 [04:34<00:16,  8.02s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 1e+03       |
|    gen/rollout/ep_rew_mean         | 1.4e+03     |
|    gen/rollout/ep_rew_wrapped_mean | 1.12e+03    |
|    gen/time/fps                    | 1599        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 286720      |
|    gen/train/approx_kl             | 0.052379884 |
|    gen/train/clip_fraction         | 0.673       |
|    gen/train/clip_range            | 0.1         |
|    gen/train/entropy_loss          | 3.33        |
|    gen/train/explained_variance    | 0.989       |
|    gen/train/learning_rate         | 0.000263    |
|    gen/train/loss                  | 0.374       |
|    gen/train/n_updates             | 170         |
|    gen/train/policy_gradient_loss  | 0.0369      |
|    gen/train/std                   | 0.156  

round:  97%|█████████▋| 35/36 [04:42<00:08,  8.03s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.41e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 1.16e+03   |
|    gen/time/fps                    | 1598       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 5          |
|    gen/time/total_timesteps        | 294912     |
|    gen/train/approx_kl             | 0.06585422 |
|    gen/train/clip_fraction         | 0.687      |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 3.31       |
|    gen/train/explained_variance    | 0.98       |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 0.231      |
|    gen/train/n_updates             | 175        |
|    gen/train/policy_gradient_loss  | 0.056      |
|    gen/train/std                   | 0.156      |
|    gen/tra

round: 100%|██████████| 36/36 [04:50<00:00,  8.07s/it]


30% 1.21e3

In [59]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [60]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: 1594.9932954500005 +/- 58.73377060063065


In [61]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 1485.1285592999998 +/- 115.30336574953647


# GAIL-PPO 10 trajectory - 10 traj BC

In [62]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_PPO_10, gail_trainer_PPO_10 = make_PPO_learner_and_trainer_10(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [63]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1681.11066565 +/- 39.92702650567257


In [64]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [65]:
gail_trainer_PPO_10.train(300000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/14 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 1e+03    |
|    gen/rollout/ep_rew_mean  | 1.46e+03 |
|    gen/time/fps             | 3009     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 6        |
|    gen/time/total_timesteps | 20480    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.55     |
|    disc/disc_acc_expert             | 0.321    |
|    disc/disc_acc_gen                | 0.778    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.689    |
|    disc/disc_proportion_expert_pred | 0.272    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1e+03    |
|    disc/n_generated                 | 1e+03    |
-

round:   7%|▋         | 1/14 [00:13<02:59, 13.81s/it]

--------------------------------------------------
| raw/                               |           |
|    gen/rollout/ep_len_mean         | 1e+03     |
|    gen/rollout/ep_rew_mean         | 1.45e+03  |
|    gen/rollout/ep_rew_wrapped_mean | 1.39e+03  |
|    gen/time/fps                    | 3134      |
|    gen/time/iterations             | 1         |
|    gen/time/time_elapsed           | 6         |
|    gen/time/total_timesteps        | 40960     |
|    gen/train/approx_kl             | 3.4752526 |
|    gen/train/clip_fraction         | 0.774     |
|    gen/train/clip_range            | 0.1       |
|    gen/train/entropy_loss          | 4.08      |
|    gen/train/explained_variance    | -0.0455   |
|    gen/train/learning_rate         | 0.000263  |
|    gen/train/loss                  | 0.547     |
|    gen/train/n_updates             | 5         |
|    gen/train/policy_gradient_loss  | 0.0736    |
|    gen/train/std                   | 0.145     |
|    gen/train/value_loss      

round:  14%|█▍        | 2/14 [00:27<02:43, 13.62s/it]

...

round:  86%|████████▌ | 12/14 [02:41<00:27, 13.51s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 1e+03      |
|    gen/rollout/ep_rew_mean         | 1.42e+03   |
|    gen/rollout/ep_rew_wrapped_mean | 912        |
|    gen/time/fps                    | 3109       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 6          |
|    gen/time/total_timesteps        | 266240     |
|    gen/train/approx_kl             | 0.09013584 |
|    gen/train/clip_fraction         | 0.728      |
|    gen/train/clip_range            | 0.1        |
|    gen/train/entropy_loss          | 3.83       |
|    gen/train/explained_variance    | 0.986      |
|    gen/train/learning_rate         | 0.000263   |
|    gen/train/loss                  | 0.147      |
|    gen/train/n_updates             | 60         |
|    gen/train/policy_gradient_loss  | 0.0651     |
|    gen/train/std                   | 0.146      |
|    gen/tra

round:  93%|█████████▎| 13/14 [02:55<00:13, 13.52s/it]

--------------------------------------------------
| raw/                               |           |
|    gen/rollout/ep_len_mean         | 1e+03     |
|    gen/rollout/ep_rew_mean         | 1.44e+03  |
|    gen/rollout/ep_rew_wrapped_mean | 968       |
|    gen/time/fps                    | 3075      |
|    gen/time/iterations             | 1         |
|    gen/time/time_elapsed           | 6         |
|    gen/time/total_timesteps        | 286720    |
|    gen/train/approx_kl             | 4.4253345 |
|    gen/train/clip_fraction         | 0.779     |
|    gen/train/clip_range            | 0.1       |
|    gen/train/entropy_loss          | 3.82      |
|    gen/train/explained_variance    | 0.949     |
|    gen/train/learning_rate         | 0.000263  |
|    gen/train/loss                  | 0.132     |
|    gen/train/n_updates             | 65        |
|    gen/train/policy_gradient_loss  | 0.0806    |
|    gen/train/std                   | 0.145     |
|    gen/train/value_loss      

round: 100%|██████████| 14/14 [03:09<00:00, 13.52s/it]


In [66]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [67]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: 1538.076227 +/- 239.7745729045172


In [68]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 1581.92845435 +/- 103.29971348663021
